In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nmrglue as ng
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [ ]:
from trentnmr import *

In [ ]:
# data_folder = '/home/tyler/data/Sep-2016-23Na/VT/*.fid'
data_folder = '/home/tylerbiggs/data/Sep-2016-23Na/VT/*.fid'
# data_folder = '/home/tyler/data/Sep-2016-23Na/23Na/27Al/*.fid'
# data_folder = '/home/tyler/data/Sep-2016-23Na/23Na/27Al/0808G1-0p15M-AlOH3-3M-NaOH-D2O/*.fid'

In [ ]:
data_files = sorted(glob.glob(data_folder))
data_files

In [ ]:
# Make a dictionary using the paths as keys, this way
# I will not lose them.
data_dict = {f: read_varian_as_nmrpipe(f) for f in data_files[0:27]}
# data_dict  # Very long output.

In [ ]:
processed_dict = {f: process_fid(value[0], value[1]) for f, value in data_dict.items()}
# processed_dict

In [ ]:
peak_dict = {f: find_nmr_peaks(value[1]) for f, value in processed_dict.items()}
# Peak list is output as:
# Locations, cluster_ids, scales, amps.
# peak_dict

In [ ]:
ppm_peak_dict = dict()
for key, value in peak_dict.items():
    # Get the corresponding uc conversion object from the processed_dict.
    uc = processed_dict[key][2]
    ppm_peak_dict[key] = {
        "center": [peak[0] for peak in value],
        "ppm_lw": [peak[2] for peak in value]
    }
    
# ppm_peak_dict

In [ ]:
for idx, item in enumerate(processed_dict.items()):
    
    # Unpack the data.
    key, value = item
    dic, data, uc = value
    
    # Get the peaks for this spectra.
    spectra_peaks = ppm_peak_dict[key]
    peak_centers = spectra_peaks['center']
    peak_lw = spectra_peaks['ppm_lw']
    
    # Create the ppm scale.
    ppm_scale = uc.ppm_scale()
    
    # Plot the nmr data.
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(ppm_scale, data, 'k', linewidth=0.75)
#     ax.set_xlim(.95, 1.5)
    ax.invert_xaxis()
    for peak, line_width in zip(peak_centers, peak_lw):
        
        # Generate the low and high values.
        # TODO: Refactor to accept functinons.
        low = int(peak - line_width)
        high = int(peak + line_width)
        
        # Ensure we did not get the order wrong.
        if low > high:
            low, high = high, low
        
        # Extract data values that make up this peak.
        peak_vals = data[low:high + 1]
        
        # Map these values to the ppm scale.
        peak_vals_ppm = ppm_scale[low:high + 1]
        
        ax.plot(
            peak_vals_ppm, 
            peak_vals.cumsum() / 8 + peak_vals.max(),
            'g-')
        
        ax.text(peak_vals_ppm[0], peak_vals.max(), peak_vals.sum() / 10000)

In [ ]:
# #   plot 0     plot 1    plot 2   plot 3
# x=[[1,2,3,4],[1,4,3,4],[1,2,3,4],[9,8,7,4]]
# y=[[3,2,3,4],[3,6,3,4],[6,7,8,9],[3,2,2,4]]

# plots = zip(x,y)
# def loop_plot(plots):
#     figs={}
#     axs={}
#     for idx,plot in enumerate(plots):
#         figs[idx]=plt.figure()
#         axs[idx]=figs[idx].add_subplot(111)
#         axs[idx].plot(plot[0],plot[1])
# return figs, axs

# figs, axs = loop_plot(plots)